<a href="https://colab.research.google.com/github/lucjalucja/analysis/blob/main/SoftSKills.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd

In [6]:
# Load data
data = pd.read_csv("Soft Skillet_ October Data Live - Activity Data vOct.csv")
add = pd.read_csv("outcome.csv")
data = pd.merge(data, add, on='Customer Account ID')
data.head()

,Activity ID,Date,Person,Call Connect,Pitch,Disposition,Customer Account ID,Outcome
0,01pElO8,9/12/2024,Owner,Connected,AI,No interest,10898,Loss
1,02cNE8F,9/3/2024,Manager,Connected,Operator,Deferred to Other Stakeholder,26600,Open
2,0498JpwY8S,9/6/2024,Owner,Connected,AI,Engaged,29419,Loss
3,06Q6UJH,9/9/2024,Manager,Connected,Operator,No Interest,97655,Loss
4,07IT4x0,9/5/2024,Manager,Connected,Smooth,No Interest,57549,Loss


In [7]:
# Filter out rows where 'person', 'pitch', and 'disposition' are NaN
filtered_data = data.dropna(subset=['Person', 'Pitch', 'Disposition'], how='all')
filtered_data.head()

,Activity ID,Date,Person,Call Connect,Pitch,Disposition,Customer Account ID,Outcome
0,01pElO8,9/12/2024,Owner,Connected,AI,No interest,10898,Loss
1,02cNE8F,9/3/2024,Manager,Connected,Operator,Deferred to Other Stakeholder,26600,Open
2,0498JpwY8S,9/6/2024,Owner,Connected,AI,Engaged,29419,Loss
3,06Q6UJH,9/9/2024,Manager,Connected,Operator,No Interest,97655,Loss
4,07IT4x0,9/5/2024,Manager,Connected,Smooth,No Interest,57549,Loss


In [8]:
# Group by 'Person' and 'Pitch' to count the number of successful dispositions
success_counts = filtered_data[filtered_data['Outcome'] == 'Open'].groupby(['Person', 'Pitch'])['Disposition'].count().reset_index(name='SuccessCount')

# Group by 'Person' and 'Pitch' to count the total number of dispositions
total_counts = filtered_data.groupby(['Person', 'Pitch'])['Disposition'].count().reset_index(name='TotalCount')

# Merge success and total counts, using 'outer' join to include all combinations
merged_counts = pd.merge(success_counts, total_counts, on=['Person', 'Pitch'], how='outer')

# Fill NaN values in 'SuccessCount' with 0 (for combinations with no successes)
merged_counts['SuccessCount'] = merged_counts['SuccessCount'].fillna(0)

# Calculate the success rate
merged_counts['SuccessRate'] = (merged_counts['SuccessCount'] / merged_counts['TotalCount']) * 100


print("Success Rates for all combinations:")
merged_counts

Success Rates for all combinations:


,Person,Pitch,SuccessCount,TotalCount,SuccessRate
0,Manager,AI,9,103,8.737864
1,Manager,Operator,78,196,39.795918
2,Manager,Smooth,14,143,9.790210
3,Owner,AI,14,68,20.588235
4,Owner,Operator,26,67,38.805970
5,Owner,Smooth,24,70,34.285714


In [ ]:
# Install necessary libraries for transcription
!pip install pydub
!pip install SpeechRecognition

In [13]:
import os
import subprocess
from pydub import AudioSegment
import speech_recognition as sr

def transcribe_audio(file_path):
  """
  Transcribes an audio file using Google Speech Recognition.

  Args:
    file_path: The path to the audio file.

  Returns:
    The transcribed text, or None if transcription fails.
  """
  recognizer = sr.Recognizer()
  try:
    # Explicitly convert to WAV using pydub
    audio = AudioSegment.from_file(file_path)
    audio.export("temp.wav", format="wav")  # Export to a temporary WAV file

    with sr.AudioFile("temp.wav") as source:
      audio_data = recognizer.record(source)
      text = recognizer.recognize_google(audio_data)

    os.remove("temp.wav")  # Remove the temporary file
    return text

  except sr.UnknownValueError:
    print(f"Google Speech Recognition could not understand audio in {file_path}")
    return None
  except sr.RequestError as e:
    print(f"Could not request results from Google Speech Recognition service; {e}")
    return None
  except Exception as e:
    print(f"An error occurred: {e}")  # Catch and print other exceptions
    return None


def convert_mov_to_wav(mov_file_path):
    """Converts .mov file to .wav using ffmpeg"""
    wav_file_path = mov_file_path.replace(".mov", ".wav")
    try:
        subprocess.run(["ffmpeg", "-i", mov_file_path, wav_file_path], check=True)
        return wav_file_path
    except subprocess.CalledProcessError as e:
        print(f"Error converting .mov to .wav: {e}")
        return None

def process_files_in_directory(directory):
    """Processes all .mov and .mp3 files in a directory"""
    for filename in os.listdir(directory):
        if filename.endswith(".mov"):
            file_path = os.path.join(directory, filename)
            wav_file_path = convert_mov_to_wav(file_path)
            if wav_file_path:
                transcription = transcribe_audio(wav_file_path)
                if transcription:
                    output_file_path = file_path.replace(".mov", ".txt")
                    with open(output_file_path, "w") as f:
                        f.write(transcription)
        elif filename.endswith(".mp3"):
            file_path = os.path.join(directory, filename)
            transcription = transcribe_audio(file_path) # transcribe_audio directly for .mp3 files
            if transcription:
                output_file_path = file_path.replace(".mp3", ".txt")
                with open(output_file_path, "w") as f:
                    f.write(transcription)


directory_to_process = "/content/additional"  # path to directory with recordings
process_files_in_directory(directory_to_process)